[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório.
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/ml_intro/class_03/Laborat%C3%B3rios/lab_utils_ml_intro_week_3.zip
!unzip -n -q lab_utils_ml_intro_week_3.zip

In [ ]:
# Testar se estamos no Google Colab
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Função de Custo para Regressão Logística

## Objetivos
Neste laboratório, você irá:
- examinar a implementação e utilizar a função de custo para regressão logística.

In [ ]:
!pip install ipympl

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import  plot_data, sigmoid, dlc
plt.style.use('./deeplearning.mplstyle')

## Conjunto de dados
Vamos começar com o mesmo conjunto de dados usado no laboratório de fronteira de decisão.

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])  #(m,n)
y_train = np.array([0, 0, 0, 1, 1, 1])                                           #(m,)

Usaremos uma função auxiliar para plotar esses dados. Os pontos de dados com rótulo $y=1$ são mostrados como cruzes vermelhas, enquanto os pontos de dados com rótulo $y=0$ são mostrados como círculos azuis.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

# Defina ambos os eixos como 0-4
ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## Função de custo

Em um laboratório anterior, você desenvolveu a função *perda logística*. Lembre-se, a perda é definida para ser aplicada a um exemplo. Aqui você combina as perdas para formar o **custo**, que inclui todos os exemplos.


Lembre-se de que para regressão logística, a função de custo tem a forma

$$ J(\mathbf{w},b) = \frac{1}{m} \sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

onde
* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ é o custo de um único ponto de dados, que é:

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$
    
*  onde m é o número de exemplos de treinamento no conjunto de dados e:
$$
\begin{align}
  f_{\mathbf{w},b}(\mathbf{x^{(i)}}) &= g(z^{(i)})\tag{3} \\
  z^{(i)} &= \mathbf{w} \cdot \mathbf{x}^{(i)}+ b\tag{4} \\
  g(z^{(i)}) &= \frac{1}{1+e^{-z^{(i)}}}\tag{5} 
\end{align}
$$
 

<a name='ex-02'></a>
#### Code Description

#### Descrição do código

O algoritmo para `compute_cost_logistic` percorre todos os exemplos calculando a perda de cada exemplo e acumulando o total.

Observe que as variáveis ​​X e y não são valores escalares, mas matrizes de forma ($m, n$) e ($𝑚$,) respectivamente, onde $𝑛$ é o número de recursos e $𝑚$ é o número de exemplos de treinamento .


In [ ]:
def compute_cost_logistic(X, y, w, b):
    """
    Calcula o custo

    Args:
      X (ndarray (m,n)): Dados, m exemplos com n recursos
      y (ndarray (m,)) : valores alvo
      w (ndarray (n,)) : parâmetros do modelo
      b (escalar)       : parâmetros do modelo
      
    Returns:
      cost (escalar): custo
    """

    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
             
    cost = cost / m
    return cost


Verifique a implementação da função de custo usando a célula abaixo.

In [ ]:
w_tmp = np.array([1,1])
b_tmp = -3
print(compute_cost_logistic(X_train, y_train, w_tmp, b_tmp))

**Saída Esperada**: 0.3668667864055175

## Exemplo
Agora, vamos ver qual é o resultado da função de custo para um valor diferente de $w$.

* Em um laboratório anterior, você traçou o limite de decisão para  $b = -3, w_0 = 1, w_1 = 1$. Ou seja, você obteve `b = -3, w = np.array([1,1])`.

* Digamos que você queira ver se $b = -4, w_0 = 1, w_1 = 1$, ou `b = -4, w = np.array([1,1])` fornece um modelo melhor.

Vamos primeiro traçar o limite de decisão para esses dois valores $b$ diferentes para ver qual deles se ajusta melhor aos dados.

* For $b = -3, w_0 = 1, w_1 = 1$, we'll plot $-3 + x_0+x_1 = 0$ (mostrado em azul)
* For $b = -4, w_0 = 1, w_1 = 1$, we'll plot $-4 + x_0+x_1 = 0$ (mostrado em magenta)

In [ ]:
import matplotlib.pyplot as plt

# Escolha valores entre 0 e 6
x0 = np.arange(0,6)

# Trace os duas fronteiras de decisão
x1 = 3 - x0
x1_other = 4 - x0

fig,ax = plt.subplots(1, 1, figsize=(4,4))
# Plote as fronteiras de decisão
ax.plot(x0,x1, c=dlc["dlblue"], label="$b$=-3")
ax.plot(x0,x1_other, c=dlc["dlmagenta"], label="$b$=-4")
ax.axis([0, 4, 0, 4])

# Plote or dados originais
plot_data(X_train,y_train,ax)
ax.axis([0, 4, 0, 4])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.legend(loc="upper right")
plt.title("Fronteira de Decisão")
plt.show()

Você pode ver neste gráfico que `b = -4, w = np.array([1,1])` é um modelo pior para os dados de treinamento. Vamos ver se a implementação da função de custo reflete isso.

In [ ]:
w_array1 = np.array([1,1])
b_1 = -3
w_array2 = np.array([1,1])
b_2 = -4

print("Custo para b = -3 : ", compute_cost_logistic(X_train, y_train, w_array1, b_1))
print("Custo para b = -4 : ", compute_cost_logistic(X_train, y_train, w_array2, b_2))

**Saída Esperada**

Custo para b = -3 :  0.3668667864055175

Custo para = -4 :  0.5036808636748461


Você pode ver que a função de custo se comporta conforme o esperado e o custo para `b = -4, w = np.array([1,1])` é realmente maior do que o custo para `b = -3, w = np.array ([1,1])`

## Parabéns!
Neste laboratório você examinou e utilizou a função de custo para regressão logística.